In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


We want to build the model off of HuggingFace libraries instead of fairseq scripts. It is easier to debug and understand what you're doing when building the model.
We are starting from absolute scratch. We need to train a sentencepiece tokenizer on joint corpus. That's kinda done in the previous notebook.

However I never got a hf model to train with the tokenizer. Let's try that.

In [9]:
from pathlib import Path
from tokenizers.implementations import SentencePieceBPETokenizer
import pandas as pd

### Load Pretrained Model from hub

In [10]:
TOKENIZER_BATCH_SIZE = 256  # Batch-size to train the tokenizer on
TOKENIZER_VOCABULARY = 25000  # Total number of unique subwords the tokenizer can have

BLOCK_SIZE = 128  # Maximum number of tokens in an input sample
NSP_PROB = 0.50  # Probability that the next sentence is the actual next sentence in NSP
SHORT_SEQ_PROB = 0.1  # Probability of generating shorter sequences to minimize the mismatch between pretraining and fine-tuning.
MAX_LENGTH = 512  # Maximum number of tokens in an input sample after padding

MLM_PROB = 0.2  # Probability with which tokens are masked in MLM

TRAIN_BATCH_SIZE = 2  # Batch-size for pretraining the model on
MAX_EPOCHS = 1  # Maximum number of epochs to train the model for
LEARNING_RATE = 1e-4  # Learning rate for training the model

MODEL_CHECKPOINT = "mbart-large-50"  # Name of pretrained model from 🤗 Model Hub

In [4]:
# Adding all the data i have in 1 large text file to train my tokenizer

# paths = [str(x) for x in Path("../experiments/data/").glob("**/*.jsonl")]
# paths_df = [pd.read_json(x,lines=True) for x in paths]

# all_str = ''
# for i in paths_df:
#     add_str = ''.join(i['input'].values + i['target'].values)
#     all_str += add_str

# sentences = all_str.split(". ")

# with open("all_sentences.txt", "w") as file:
#     for sentence in sentences:
#         file.write(sentence + ".\n")

# !spm_train --input="/mnt/disk/yrajcoomar/kreol-benchmark/pipelines/all_sentences.txt" --model_prefix=kreol --vocab_size=20000 --model_type=bpe

In [5]:
## Train tokenizer

# def batch_iterator():
#     for i in range(0, len(all_str), TOKENIZER_BATCH_SIZE):
#         yield all_str[i : i + TOKENIZER_BATCH_SIZE]

# tokenizer = SentencePieceBPETokenizer()
# tokenizer.train_from_iterator(batch_iterator(), vocab_size=TOKENIZER_BATCH_SIZE, min_frequency=2, special_tokens=[
#     "<s>",
#     "<pad>",
#     "</s>",
#     "<unk>",
#     "<mask>",
# ])


# !mkdir tok
# tokenizer.save_model("./tok/")

In [6]:
## idrk whats going on

# from tokenizers.implementations import SentencePieceBPETokenizer
# from tokenizers.processors import BertProcessing


# tokenizer = SentencePieceBPETokenizer(
#     "tokenizer/vocab.json",
#     "tokenizer/merges.txt",
# )
# tokenizer._tokenizer.post_processor = BertProcessing(
#     ("</s>", tokenizer.token_to_id("</s>")),
#     ("<s>", tokenizer.token_to_id("<s>")),
# )
# tokenizer.enable_truncation(max_length=512)

In [14]:
from transformers import MBart50Tokenizer

tokenizer = MBart50Tokenizer.from_pretrained("/mnt/disk/yrajcoomar/kreol-benchmark/pipelines/tok",max_len=256)

In [ ]:
tokenizer("kifr")['input_ids']

[20004, 35, 19939, 19929, 2]

In [ ]:
tokenizer.decode([20004])

'en_XX'

In [ ]:
tokenizer.decode(tokenizer("kifr")['input_ids'])

'en_XX kifr</s>'

In [ ]:
tokenizer.bos_token = "<bos>"


In [ ]:
tokenizer.unk_token_id

3

In [ ]:
tokenizer.bos_token_id

3

In [ ]:
tokenizer

MBart50Tokenizer(name_or_path='/teamspace/studios/this_studio/kreol-benchmark/pipelines/tok/', vocab_size=20054, model_max_length=256, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<bos>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>', 'additional_special_tokens': ['ar_AR', 'cs_CZ', 'de_DE', 'en_XX', 'es_XX', 'et_EE', 'fi_FI', 'fr_XX', 'gu_IN', 'hi_IN', 'it_IT', 'ja_XX', 'kk_KZ', 'ko_KR', 'lt_LT', 'lv_LV', 'my_MM', 'ne_NP', 'nl_XX', 'ro_RO', 'ru_RU', 'si_LK', 'tr_TR', 'vi_VN', 'zh_CN', 'af_ZA', 'az_AZ', 'bn_IN', 'fa_IR', 'he_IL', 'hr_HR', 'id_ID', 'ka_GE', 'km_KH', 'mk_MK', 'ml_IN', 'mn_MN', 'mr_IN', 'pl_PL', 'ps_AF', 'pt_XX', 'sv_SE', 'sw_KE', 'ta_IN', 'te_IN', 'th_TH', 'tl_XX', 'uk_UA', 'ur_PK', 'xh_ZA', 'gl_ES', 'sl_SI']}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normal

### Preprocess and split Data

In [ ]:
from datasets import load_dataset

In [ ]:
#/teamspace/studios/this_studio/kreol-benchmark/experiments/data

In [ ]:
!pwd

/teamspace/studios/this_studio


In [16]:
from datasets import load_dataset

dataset = load_dataset(
    "json",
    data_files={'train':'/mnt/disk/yrajcoomar/kreol-benchmark/experiments/data/en-cr/en-cr_train.jsonl','test':'/mnt/disk/yrajcoomar/kreol-benchmark/experiments/data/en-cr/en-cr_test.jsonl',
                'val':'/mnt/disk/yrajcoomar/kreol-benchmark/experiments/data/en-cr/en-cr_dev.jsonl'}
)


Using custom data configuration default-6c21299dff136ec5


Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 2451.85it/s]


Dataset json downloaded and prepared to /home/ubuntu/.cache/huggingface/datasets/json/default-6c21299dff136ec5/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 1331.53it/s]


In [17]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'target'],
        num_rows: 21810
    })
    test: Dataset({
        features: ['input', 'target'],
        num_rows: 1000
    })
    val: Dataset({
        features: ['input', 'target'],
        num_rows: 500
    })
})

In [40]:
def preprocess_function(examples):
    inputs = examples['input']
    outputs = examples['target']
    input_tokenized = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    output_tokenized = tokenizer(outputs, max_length=128, truncation=True, padding="max_length")
    input_ids = input_tokenized["input_ids"]
    attention_mask = input_tokenized["attention_mask"]
    decoder_input_ids = output_tokenized["input_ids"]
    decoder_attention_mask = output_tokenized["attention_mask"]
    labels = decoder_input_ids.copy()
    # Set the labels to -100 for padding tokens
    labels = [[-100 if token == tokenizer.pad_token_id else token for token in seq] for seq in labels]
    return {"input_ids": input_ids, "attention_mask": attention_mask, "decoder_input_ids": decoder_input_ids, "decoder_attention_mask": decoder_attention_mask, "labels": labels}

dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/21810 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [41]:
train_dataset = dataset['train']
test_dataset = dataset['test']
val_dataset = dataset['val']

In [43]:
train_dataset['input_ids'][0]

[20004,
 71,
 5048,
 30,
 7981,
 2829,
 88,
 14,
 525,
 85,
 13011,
 74,
 30,
 12955,
 19974,
 19930,
 15218,
 331,
 458,
 88,
 19940,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

### Training

In [44]:
import torch
torch.cuda.is_available() ## Current issue with cuda, will seek to resolve.

False

In [45]:
from transformers import MBartConfig, MBartForConditionalGeneration, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments #, Seq2SeqTrainer

config = MBartConfig(vocab_size=TOKENIZER_VOCABULARY,max_position_embeddings=512)
model = MBartForConditionalGeneration(config)


collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer, model=model
)

In [46]:
# from torch.utils.data import IterableDataset

In [47]:
from transformers import Seq2SeqTrainer

In [48]:
training_args = Seq2SeqTrainingArguments(
    output_dir='./checkpoint',
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    per_gpu_eval_batch_size=4,
    save_steps=2,
    save_total_limit=2,
    prediction_loss_only=True,
)

In [49]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


In [50]:
train_dataset

Dataset({
    features: ['input', 'target', 'input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask', 'labels'],
    num_rows: 21810
})

### UPDATELOG:
- 21/02/24 - Set up whole pipeline. Issue is cuda is only 9.1 on sv. so i can only use cudatoolkit 9.0 and torch==1.1
- 22/02/24 - Got rid of a bunch of bugs in the past week, some attention mask shape issue is up
- 27/02/24 - All issues resolved, we got it to train!!
- 29/02/24 - We trained. We got a BLEU of 4 on 50 epochs. (Baseline is 9) Set 100 epochs to train.
- 29/02/24 - BLEU 5 on 100 epochs. 

### TODO:
- Clean repo before merging in master. Purge fairseq stuff.

- Set up WandB for model eval and all that shit

- Visit possible args to train model better. Ideas are vocabulary size, label_smoothing, weight decay, adam betas, dropout, attention dropout.



In [51]:
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


: 

: 

In [9]:
model.num_parameters()
#380M

379375616

In [10]:
from transformers import LineByLineTextDataset

In [11]:

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./all_sentences.txt",
    block_size=128,
)

/home/ubuntu/miniconda3/envs/cuda_kreol/lib/python3.7/site-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


### Load Dataset

In [13]:
import pandas as pd
import os

def read_concat_jsonl(files,home_dir):
    dfs = [pd.read_json(os.path.join(home_dir,file), lines=True) for file in files]
    return pd.concat(dfs)

def concatenate_language_pairs(language_pairs_paths,home_dir):
    concatenated_dfs = {}
    for key, paths in language_pairs_paths.items():
        dfs = [read_concat_jsonl(paths,home_dir)]
        concatenated_dfs[key] = pd.concat(dfs)
    return concatenated_dfs

In [14]:
language_pairs_paths ={
    'en_cr': ['kreol-benchmark\experiments\data\en-cr\en-cr_dev.jsonl','kreol-benchmark\experiments\data\en-cr\en-cr_train.jsonl','kreol-benchmark\experiments\data\en-cr\en-cr_test.jsonl'],
    'cr':['kreol-benchmark\experiments\data\cr\cr_dev.jsonl','kreol-benchmark\experiments\data\cr\cr_train.jsonl','kreol-benchmark\experiments\data\cr\cr_test.jsonl']
}

In [15]:
data_all_dict = concatenate_language_pairs(language_pairs_paths,home_dir=r'C:\Users\yush\OneDrive\Desktop\papers')

In [16]:
data_all_dict

{'en_cr':                                                  input  \
 0    I did not come to do away with them, but to gi...   
 1    The fact is, at the time, you had to pay the t...   
 2    Angina can be described as a discomfort, heavi...   
 3             The boy said he would, but he didn't go.   
 4     Was it God in heaven or merely some human being?   
 ..                                                 ...   
 995  Any kingdom where people fight each other will...   
 996  And I am not good enough even to stoop down an...   
 997  Who among you, if your son asks for bread, you...   
 998  If that person listens, you have won back a fo...   
 999  Then he pointed to his disciples and said, the...   
 
                                                 target  
 0    Mo pa finn vini pou aboli me pou donn zot zot ...  
 1    Anverite sa lepok la pou al lekol ti ena enn f...  
 2    Nou capav dekrir anzinn couma enn sensasion in...  
 3    Garson-la reponn wi papa, li pou ale me li 